### Task: Given 26th March 2021 demand - clean dataset based on business team rules; we need to forecast demand for first 3 (30mins internal) demand value of 27th March 2021.

Author: [Shaurya Uppal](https://www.linkedin.com/in/shaurya-uppal/)

In [2]:
import pandas as pd
import numpy as np
from joblib import load, dump
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt, ceil, floor
from datetime import datetime, timedelta

In [3]:
def round_timestamp_30interval(x):
    if type(x)==str:
        x = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    return x- timedelta(minutes=x.minute%30, seconds=x.second, microseconds=x.microsecond)

def time_features(data):
    data['mins'] = data.ts.dt.minute
    data['hour'] = data.ts.dt.hour
    data['day'] = data.ts.dt.day
    data['month'] = data.ts.dt.month
    data['dayofweek'] = data.ts.dt.dayofweek
    data['quarter'] = data.ts.dt.quarter
    return data

def prediction_without_lag(df):
    return predict_without_lag.predict(df[['pickup_cluster','mins','hour','month','quarter','dayofweek']])

def prediction_with_lag(df):
    return predict_with_lag.predict(df[['pickup_cluster', 'mins', 'hour', 'month', 'quarter',
           'dayofweek', 'lag_1', 'lag_2', 'lag_3','rolling_mean']])

def shift_with_lag_and_rollingmean(df):
    df = df.sort_values(by=['pickup_cluster', 'ts']).drop_duplicates(subset=['ts','pickup_cluster'])
    df = df.set_index(['ts', 'pickup_cluster', 'mins', 'hour', 'month', 'quarter', 'dayofweek'])
    df['lag_1'] = df.groupby(level=['pickup_cluster'])['request_count'].shift(1)
    df['lag_2'] = df.groupby(level=['pickup_cluster'])['request_count'].shift(2)
    df['lag_3'] = df.groupby(level=['pickup_cluster'])['request_count'].shift(3)
    df['rolling_mean'] = df.groupby(level=['pickup_cluster'])['request_count'].apply(lambda x: x.rolling(window = 3).mean()).shift(1)

    df = df.reset_index(drop = False).dropna()
    df = df[['ts', 'pickup_cluster', 'mins', 'hour', 'month', 'quarter',
           'dayofweek', 'lag_1', 'lag_2', 'lag_3','rolling_mean','request_count']]
    return df


In [4]:

df = pd.read_csv('../Data/test_dataset/cleaned_test_booking_data.csv', compression = 'gzip', low_memory=False)
cluster_model = load('../Model/pickup_cluster_model.joblib')
predict_without_lag = load('E:\PROJECTpro\PROJECTS\project_3_demand_prediction\Project3_OLABikes\modular_code\output\prediction_model_without_lag_1.joblib')
predict_with_lag = load('E:\PROJECTpro\PROJECTS\project_3_demand_prediction\Project3_OLABikes\modular_code\output\prediction_model_with_lag.joblib')


C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MiniBatchKMeans from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


### Use Clustering Kmeans Model for Geospacial Feature - `pickup_cluster`

In [5]:
df['pickup_cluster'] = cluster_model.predict(df[['pick_lat','pick_lng']])
df.head(10)

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,pickup_cluster
0,2021-03-26 06:49:38,-1,12.903468,77.637080,12.916259,77.675476,31
1,2021-03-26 15:14:23,0,12.903838,77.591774,12.890039,77.593720,24
2,2021-03-26 15:57:32,6,12.963516,77.674740,12.912828,77.627310,32
3,2021-03-26 23:34:53,7,12.944017,77.564270,12.967625,77.608060,45
4,2021-03-26 23:45:56,9,12.983270,77.752070,12.963221,77.748400,41
5,2021-03-26 18:54:05,11,12.919469,77.671100,12.933288,77.607310,49
6,2021-03-26 18:42:49,15,12.947335,77.684310,12.974627,77.606064,0
7,2021-03-26 23:14:56,15,12.979332,77.640590,12.947475,77.684230,27
8,2021-03-26 10:59:13,17,12.923716,77.607410,12.922842,77.593240,18
9,2021-03-26 16:44:09,53,12.888448,77.577240,12.937987,77.568726,11


### Data preparation and processing

In [6]:
df['ts'] = np.vectorize(round_timestamp_30interval)(df['ts'])
df['ts'] = pd.to_datetime(df['ts'])

df = df[['ts','number','pickup_cluster']]
df=df.groupby(by = ['ts','pickup_cluster']).count().reset_index()
df.columns = ['ts','pickup_cluster','request_count']

## Adding Dummy pickup cluster -1

## Change this Data based on your data
l = [datetime(2021,3,26,00,00,00) + timedelta(minutes = 30*i) for i in range(0,51)]
lt = []
for x in l:
    lt.append([x, -1, 0])
temp = pd.DataFrame(lt, columns = ['ts','pickup_cluster','request_count'])
df = df.append(temp,ignore_index=True)

data = df.set_index(['ts', 'pickup_cluster']).unstack().fillna(value=0).asfreq(freq='30Min').stack().sort_index(level=1).reset_index()

# Removing Dummy Cluster
data = data[data.pickup_cluster>=0]

df = time_features(data)

### Model without Lag (past data) requirement

In [7]:
data_without_lag = df[df['ts']>=datetime(2021,3,27,00,00,00)].__copy__()
data_without_lag['request_count'] = prediction_without_lag(data_without_lag)
data_without_lag

C:\Users\LENOVO\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


,ts,pickup_cluster,request_count,mins,hour,day,month,dayofweek,quarter
99,2021-03-27 00:00:00,0,6.871499,0,0,27,3,5,1
100,2021-03-27 00:30:00,0,3.866220,30,0,27,3,5,1
101,2021-03-27 01:00:00,0,1.812883,0,1,27,3,5,1
150,2021-03-27 00:00:00,1,1.014596,0,0,27,3,5,1
151,2021-03-27 00:30:00,1,0.651106,30,0,27,3,5,1
...,...,...,...,...,...,...,...,...,...
2395,2021-03-27 00:30:00,48,3.350918,30,0,27,3,5,1
2396,2021-03-27 01:00:00,48,1.508002,0,1,27,3,5,1
2445,2021-03-27 00:00:00,49,4.490405,0,0,27,3,5,1
2446,2021-03-27 00:30:00,49,3.047003,30,0,27,3,5,1


In [8]:
data_without_lag.to_csv('../Data/test_dataset_prediction_output/prediction_without_lag_model.csv',index = False, compression = 'gzip')

### Using Iteration 3 - Best Model with Lag Features and Rolling Means (Recursive Multi-Step Forecast used)

In [9]:
start_date = datetime(2021,3,27,00,00,00) 
for x in range(3):
    df = shift_with_lag_and_rollingmean(df)
    df.loc[df[df['ts']==start_date+timedelta(minutes=30*x)].index,'request_count'] = prediction_with_lag(df[df['ts']==start_date+timedelta(minutes=30*x)])

C:\Users\LENOVO\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\LENOVO\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\LENOVO\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [10]:
data_with_lag = df[df['ts']>=datetime(2021,3,27,00,00,00)].__copy__()
data_with_lag

,ts,pickup_cluster,mins,hour,month,quarter,dayofweek,lag_1,lag_2,lag_3,rolling_mean,request_count
42,2021-03-27 00:00:00,0,0,0,3,1,5,8.000000,14.000000,19.0,13.666667,5.236571
43,2021-03-27 00:30:00,0,30,0,3,1,5,5.236571,8.000000,14.0,9.078857,3.354270
44,2021-03-27 01:00:00,0,0,1,3,1,5,3.354270,5.236571,8.0,5.530281,1.969474
87,2021-03-27 00:00:00,1,0,0,3,1,5,1.000000,3.000000,4.0,2.666667,0.762030
88,2021-03-27 00:30:00,1,30,0,3,1,5,0.762030,1.000000,3.0,1.587343,0.442128
...,...,...,...,...,...,...,...,...,...,...,...,...
2068,2021-03-27 00:30:00,48,30,0,3,1,5,4.388031,5.000000,11.0,6.796010,2.704878
2069,2021-03-27 01:00:00,48,0,1,3,1,5,2.704878,4.388031,5.0,4.030970,1.651302
2112,2021-03-27 00:00:00,49,0,0,3,1,5,7.000000,10.000000,13.0,10.000000,4.195784
2113,2021-03-27 00:30:00,49,30,0,3,1,5,4.195784,7.000000,10.0,7.065261,2.776667


In [11]:
data_with_lag.to_csv('../Data/test_dataset_prediction_output/prediction_with_lag_model.csv',index = False, compression = 'gzip')

### Thank You for Choosing this Project. Hope you liked it. Check our Scripts Folder for Modularized py scripts. 